In [ ]:
#Step1 - Generate SAX
#Stepe - Generate BagOfWords

# Configuration Data
SELECT_DATE = "20210120"
SAX_BINS = 5
SAX_STRATEGY ='uniform'
BOW_WINDOW_SIZE = 1
BOW_NUM_REDUCE = True

In [ ]:
import os
import pandas as pd
import numpy as np 
from pyts.bag_of_words import WordExtractor
from pyts.approximation import SymbolicAggregateApproximation

header_list = ['Date', 'DateStr','TimeStr' ,'Open', 'High', 'Low', 'Close', 'Null1', 'Null2', 'Null3', 'Null4']
path = r'/home/parag/devArea/pyStock/Data/Consolidated/*banknifty.txt'
BNdf = pd.concat((pd.read_csv(f, header=None, sep=',', names=header_list) for f in iglob(path, recursive=True)), ignore_index=True)
BNdf['Date']=(pd.to_datetime(BNdf.DateStr.astype(str) + ' ' + BNdf.TimeStr.astype(str), format='%Y%m%d %H:%M'))

BNdf = BNdf.drop(['DateStr', 'TimeStr', 'Null1', 'Null2', 'Null3', 'Null4'], axis=1)
BNdf = BNdf.drop(['Open', 'High', 'Low'], axis=1)
BNdf.set_index('Date',inplace=True)

In [ ]:
# Filter Date & Read day open 1st candle
dayFiltered = BNdf.loc[SELECT_DATE]
dayOpen = dayFiltered["Close"].iloc[0]

# Convert to array and substract dayOpen to normalize the data, else SAX fails
arr = (dayFiltered['Close'] - dayOpen).round().to_list()
X = arr


In [ ]:
# SAX transformation
sax = SymbolicAggregateApproximation(n_bins=SAX_BINS, strategy=SAX_STRATEGY)
X_sax = sax.fit_transform([arr])

print("SAX Normalisation of length ",len(X_sax[0]), "\n", X_sax)

In [ ]:
bow = WordExtractor(window_size=BOW_WINDOW_SIZE, numerosity_reduction=BOW_NUM_REDUCE)
X_bow = bow.transform(X_sax)
print("Bag of Words Extraction -", len(X_bow[0]),"\n" ,X_bow)

In [ ]:
#print(X_bow, "is of type", type(X_bow))